<h1>
    <b>
        <center>
            Tutorial de Chatbot Transformer con Tensorflow 2.0
        </center>
    </b>
</h1>

<h6>
    Pueden ver el  tutorial  en su idioma orginal <A href="https://github.com/Abonia1/TF-Chatbot">aquí</A>
</h6>

<h2>Pasos Previos al tutorial</h2>
<ol type="A">
<li>Descargar el dataset de cornell_movie_dialogs <A href="https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html">aquí</A></li>
<li>Descomprimirlo el archivo .zip</li>
<li>ubicarlo en la carpeta "Chatbot".</li>
<ol>

In [ ]:
import tensorflow as tf
assert tf.__version__.startswith('2')
tf.random.set_seed(1234)

#!pip install tensorflow-datasets==1.2.0
import tensorflow_datasets as tfds

import os
import re
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
path_lines = "cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_lines.txt"

path_movie_conversions = (
    "cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_conversations.txt"
)

In [ ]:
MAX_SAMPLES = 50000

def preprocess_sentences(frase):
    frase = frase.lower().strip()
    frase = re.sub(r"([?.!,])", r"\1 ", frase)
    frase = re.sub(r'[" "]+', " ", frase)
    frase = re.sub(r"[^a-zA-Z?.!,]+", " ", frase)
    frase = frase.strip()
    return frase

def load_sentence():
    id2line = {}
    with open(path_lines, errors="ignore") as file:
        lines = file.readlines()
    for line in lines:
        parts = line.replace("\n", "").split(" +++$+++ ")
        id2line[parts[0]] = parts[4]

    inputs, outputs = [], []
    with open(path_movie_conversions, "r") as file:
        lines = file.readlines()
    for line in lines:
        parts = line.replace("\n", "").split(" +++$+++ ")
        conversation = [line[1:-1] for line in parts[3][1:-1].split(", ")]
        for i in range(len(conversation) - 1):
            inputs.append(preprocess_sentences(id2line[conversation[i]]))
            outputs.append(preprocess_sentences(id2line[conversation[i + 1]]))
            if len(inputs) >= MAX_SAMPLES:
                return inputs, outputs
    return inputs, outputs

In [ ]:
inputs, outputs = load_sentence()

In [ ]:
tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    inputs + outputs, target_vocab_size=2**13)

In [ ]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
print(tokenizer.encode("hello there!!"))
print(tokenizer.decode(tokenizer.encode("hello there!!")))
print(tokenizer.vocab_size)

[2276, 126, 8110, 8110]
hello there!!
8333


In [ ]:
print(START_TOKEN, END_TOKEN)

[8333] [8334]


```
lista  = [2]+ [1, 2, 4] + [34]
print(lista)
>> [2, 1, 2, 4, 34]
```

In [ ]:
def tokenize_and_filter(input, output, MAX_LENGHT):
    tokenize_inputs, tokienize_outputs = [], []
    for(sentence1, sentence2) in zip(input, output):
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
        if len(sentence1) <= MAX_LENGHT and len(sentence2) <= MAX_LENGHT:
            tokenize_inputs.append(sentence1)
            tokienize_outputs.append(sentence2)
    tokenize_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenize_inputs, maxlen= MAX_LENGHT, padding= 'post')
    tokienize_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokienize_outputs, maxlen= MAX_LENGHT, padding= 'post')
    return tokenize_inputs, tokienize_outputs

In [ ]:
inputs, outputs = tokenize_and_filter(inputs, outputs, 40)

In [ ]:
print(inputs.shape)

(44097, 40)


<h2>Creando dataset con tf.data.Datasets<br></h2>

Pueden revisar la documentación de tensorflow [aquí](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': inputs,
        'dec_inputs': outputs[:, :-1]
    },
    {
        'outputs': outputs[:,1:]
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
dataset

<PrefetchDataset shapes: ({inputs: (None, 40), dec_inputs: (None, 39)}, {outputs: (None, 39)}), types: ({inputs: tf.int32, dec_inputs: tf.int32}, {outputs: tf.int32})>

<h2> <b>Attention</b> </h2>

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # add the mask zero out padding tokens.
  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)

  return tf.matmul(attention_weights, value)

<h2><b>MultiHeadAttetion</b></h2>

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # linear layers
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # split heads
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

<h2><b>Positonal enconding</b> </h2>

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
    # apply sin to even index in the array
    sines = tf.math.sin(angle_rads[:, 0::2])
    # apply cos to odd index in the array
    cosines = tf.math.cos(angle_rads[:, 1::2])

    pos_encoding = tf.concat([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

<h2><b>Transformer con la API fuctional</b></h2>

<h2><b>Encoding Layer</b></h2>


In [ ]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

<h2><b>Encoder</b></h2>


In [ ]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

<h2><b>Decoder Layer</b></h2>


In [ ]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

<h2></b>Decoder</b></h2>

In [ ]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

<h2><b>Transformer</h2></>

In [ ]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
  # Enmascaramos los futuros tokens para la entrada del decoder para el primer bloque de       # atención.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
  # Enmascarmos la salida encoders para el segundo bloque de atención 
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[inputs, enc_padding_mask])

  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1
MAX_LENGTH = 40
model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:

class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:

learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # ensure labels have shape (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

In [ ]:
model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 15

In [ ]:
model.fit(dataset, epochs=EPOCHS)

Epoch 1/15
690/690 [==============================] - 789s 1s/step - loss: 2.1101 - accuracy: 0.0431
Epoch 2/15
690/690 [==============================] - 677s 981ms/step - loss: 1.5062 - accuracy: 0.0784
Epoch 3/15
690/690 [==============================] - 633s 918ms/step - loss: 1.4003 - accuracy: 0.0857
Epoch 4/15
690/690 [==============================] - 634s 918ms/step - loss: 1.3411 - accuracy: 0.0901
Epoch 5/15
690/690 [==============================] - 635s 920ms/step - loss: 1.2892 - accuracy: 0.0942
Epoch 6/15
690/690 [==============================] - 715s 1s/step - loss: 1.2432 - accuracy: 0.0973
Epoch 7/15
690/690 [==============================] - 813s 1s/step - loss: 1.1879 - accuracy: 0.1017
Epoch 8/15
690/690 [==============================] - 720s 1s/step - loss: 1.1284 - accuracy: 0.1068
Epoch 9/15
690/690 [==============================] - 628s 911ms/step - loss: 1.0721 - accuracy: 0.1122
Epoch 10/15
690/690 [==============================] - 634s 918ms/step - los

In [ ]:

def evaluate(sentence):
  sentence = preprocess_sentences(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False) 
    # Seleccionamos la última palabra de la dimesión seq_len
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # Retornamos el resultado si  predicted_id es igual que el token final.
    if tf.equal(predicted_id, END_TOKEN[0]):
        break

    # Concatenamos el predicted_id a la salida que se le da al decodificador
   # como input.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)


def predict(sentence, name):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('{}: {}'.format(name, sentence))
  print('Bot: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
output = predict('Hello', 'Alex')

Alex: Hello
Bot: hi.


In [ ]:
model.save_weights('path_to_my_model.h5')

In [ ]:
model.load_weights('path_to_my_model.h5')

In [ ]:
output = predict('Where have you been?', "Alex")

Alex: Where have you been?
Bot: i m a little scared of myself.


In [ ]:
def dialog(name):
    while(True):
        sentence = input('Write a sentences')
        if sentence.lower() == "f":
            break
        predict(sentence, name)

In [ ]:
dialog("Alex")

Alex: hello
Bot: hi.
Alex: how are you?
Bot: i m sorry i didn t have to do anything. i want to be unzip my dress.
Alex: do you study today?
Bot: i m going to be a little girl.
Alex: tell me about you
Bot: you re not going to be late.
Alex: ok, ok 
Bot: you re a fucking asshole! you re a fucking bull to you!
Alex: calm down, please
Bot: i m going to give you something.
Alex: ok, see you
Bot: okay.
Alex: good bye 
Bot: thank you.
